# Ramp of the simulation

In [2]:
#-----Importa paquetes escenciales

from __future__ import division
import hoomd
import hoomd.md

#-----Define las variables importantes

t_prev = 8.5
t_new = 9.0
pressure = 0.1

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_8.5_trayectoria_estable.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

#----Integra usando un NPT

temp = hoomd.variant.linear_interp(points = [(0,t_prev), (5e4, t_new)])
npt = hoomd.md.integrate.npt(group = mesogens, kT = temp, tau = 2.0, tauP = 1.0, P = pressure);

#-----Guarda la información de la simulación en un archivo .log y en un .gsd

log_file = "T_" + str(t_new) + "_" + str(pressure) + "_ramp.log"
gsd_file = "T_" + str(t_new) + "_" + str(pressure) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_new) + "_" + str(pressure) + "_ramp.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite=True); 

notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "rigid_center" created containing 1000 particles


In [3]:
hoomd.run(5e4)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 1777503 / 1827000 | TPS 50.188 | ETA 00:16:26
Time 00:00:20 | Step 1777951 / 1827000 | TPS 44.6555 | ETA 00:18:18
Time 00:00:30 | Step 1778435 / 1827000 | TPS 48.3807 | ETA 00:16:43
Time 00:00:40 | Step 1778919 / 1827000 | TPS 48.374 | ETA 00:16:33
Time 00:00:50 | Step 1779461 / 1827000 | TPS 54.0911 | ETA 00:14:38
Time 00:01:00 | Step 1780028 / 1827000 | TPS 56.6879 | ETA 00:13:48
Time 00:01:10 | Step 1780629 / 1827000 | TPS 59.9805 | ETA 00:12:53
Time 00:01:20 | Step 1781209 / 1827000 | TPS 57.9763 | ETA 00:13:09
Time 00:01:30 | Step 1781763 / 1827000 | TPS 55.2974 | ETA 00:13:38
Time 00:01:40 | Step 1782327 / 1827000 | TPS 56.3056 | ETA 00:13:13
Time 00:01:50 | Step 1782923 / 1827000 | TPS 59.5321 | ETA 00:12:20
Time 00:02: